# Pandas Merge and GroupBy Walkthrough

In [1]:
import numpy as np
import pandas as pd

In [2]:
dates = pd.date_range('20130101', periods=6)
df = pd.DataFrame(np.random.randn(6,4), index=dates, columns=list('ABCD'))
df2 = pd.DataFrame({
    'A': 1.,
    'B': pd.Timestamp('20130102'),
    'C': pd.Series(1, index=list(range(4)), dtype='float32'),
    'D': np.array([3]*4, dtype='int32'),
    'E': pd.Categorical(['test', 'train', 'test', 'train']),
    'F': 'foo'
})

### Merge

In [3]:
df = pd.DataFrame(np.random.randn(10,4))
df

,0,1,2,3
0,-0.776774,-1.344645,-0.287032,-2.742271
1,-0.811495,0.668777,-0.473446,0.030516
2,-0.900136,-0.368832,1.127883,-1.928775
3,-0.383631,-0.812882,-0.670257,0.275341
4,-0.981496,-1.193950,-0.114997,1.201669
5,-1.550951,0.147977,0.158484,0.422477
6,1.409685,-0.689267,0.725356,-0.274832
7,-0.460677,-1.717361,-0.272976,0.636133
8,2.437469,0.169602,0.228513,0.052846
9,-1.074857,0.637518,0.215382,0.523746


In [4]:
pieces = [df[:3],df[3:7],df[7:]]
pd.concat(pieces)

,0,1,2,3
0,-0.776774,-1.344645,-0.287032,-2.742271
1,-0.811495,0.668777,-0.473446,0.030516
2,-0.900136,-0.368832,1.127883,-1.928775
3,-0.383631,-0.812882,-0.670257,0.275341
4,-0.981496,-1.193950,-0.114997,1.201669
5,-1.550951,0.147977,0.158484,0.422477
6,1.409685,-0.689267,0.725356,-0.274832
7,-0.460677,-1.717361,-0.272976,0.636133
8,2.437469,0.169602,0.228513,0.052846
9,-1.074857,0.637518,0.215382,0.523746


In [5]:
left = pd.DataFrame({'key': ['foo', 'bar'], 'lval': [1, 2]})
right = pd.DataFrame({'key': ['foo', 'bar'], 'rval': [4, 5]})

In [6]:
pd.merge(left, right, on='key')

,key,lval,rval
0,foo,1,4
1,bar,2,5


In [8]:
pd.merge(left, right, on='key', how='outer')

,key,lval,rval
0,foo,1,4
1,bar,2,5


### Grouping

In [9]:
df = pd.DataFrame({'A': ['foo', 'bar', 'foo', 'bar',
                             'foo', 'bar', 'foo', 'foo'],
                        'B': ['one', 'one', 'two', 'three',
                          'two', 'two', 'one', 'three'],
                       'C': np.random.randn(8),
                       'D': np.random.randn(8)})

In [10]:
df.groupby('A').sum()

,C,D
A,,
bar,-3.561473,-1.575868
foo,0.079931,0.769901


In [11]:
df.groupby(['A','B']).sum()

C         D
A   B                        
bar one   -0.097239 -0.164341
    three -0.810308 -0.992733
    two   -2.653927 -0.418794
foo one   -0.374421 -0.243635
    three -0.916439  0.906595
    two    1.370791  0.106941

In [12]:
df.groupby('A').agg({'C': np.sum, 'D': np.max})

,C,D
A,,
bar,-3.561473,-0.164341
foo,0.079931,0.906595


# Stacking

### Stack

In [14]:
tuples = list(zip(*[['bar', 'bar', 'baz', 'baz',
                         'foo', 'foo', 'qux', 'qux'],
                        ['one', 'two', 'one', 'two',
                         'one', 'two', 'one', 'two']]))
index = pd.MultiIndex.from_tuples(tuples, names=['first','second'])
df = pd.DataFrame(np.random.randn(8,2), index=index, columns=['A','B'])
df2 = df[:4]
df2

A         B
first second                    
bar   one     0.156147 -0.041290
      two     1.976329  0.118863
baz   one    -0.416222  1.312006
      two     0.843704  0.527548

In [17]:
stacked = df2.stack()
stacked

first  second   
bar    one     A    0.156147
               B   -0.041290
       two     A    1.976329
               B    0.118863
baz    one     A   -0.416222
               B    1.312006
       two     A    0.843704
               B    0.527548
dtype: float64

In [18]:
stacked.unstack()

A         B
first second                    
bar   one     0.156147 -0.041290
      two     1.976329  0.118863
baz   one    -0.416222  1.312006
      two     0.843704  0.527548

In [19]:
stacked.unstack(1)

second        one       two
first                      
bar   A  0.156147  1.976329
      B -0.041290  0.118863
baz   A -0.416222  0.843704
      B  1.312006  0.527548

In [20]:
stacked.unstack(0)

first          bar       baz
second                      
one    A  0.156147 -0.416222
       B -0.041290  1.312006
two    A  1.976329  0.843704
       B  0.118863  0.527548

### Pivot Tables

In [21]:
df = pd.DataFrame({'A': ['one', 'one', 'two', 'three'] * 3,
                       'B': ['A', 'B', 'C'] * 4,
                       'C': ['foo', 'foo', 'foo', 'bar', 'bar', 'bar'] * 2,
                       'D': np.random.randn(12),
                       'E': np.random.randn(12)})

In [23]:
pd.pivot_table(df, values='D', index=['A','B'], columns=['C'])

C             bar       foo
A     B                    
one   A -0.009834  0.626536
      B -2.028801 -2.076257
      C -0.946111  0.299265
three A  0.105755       NaN
      B       NaN -1.295496
      C  0.047240       NaN
two   A       NaN -0.220187
      B  0.463923       NaN
      C       NaN  0.579405